# Import netflow sequence and box encoding

In [1]:
import csv

infile = "data/flows_for_rnn_model_100000.csv"

fp = open(infile, 'r')
flowdata = [[int(x) for x in line] for line in csv.reader(fp)]
fp.close()

flowdata[:10]

[[1, 0, 9501, 138, 2205, 122, 1, 3, 12],
 [1, 0, 2244, 122, 8986, 122, 1, 2, 9],
 [1, 0, 2244, 122, 8986, 122, 1, 2, 9],
 [1, 0, 5354, 122, 9484, 0, 1, 0, 5],
 [1, 0, 5354, 122, 9484, 0, 1, 0, 5],
 [1, 0, 5354, 122, 9484, 0, 1, 0, 5],
 [1, 0, 8986, 122, 2244, 122, 1, 2, 8],
 [1, 0, 8986, 122, 2244, 122, 1, 2, 8],
 [1, 0, 8986, 122, 2408, 122, 1, 2, 8],
 [1, 0, 8986, 122, 2408, 122, 1, 2, 8]]

Note that the schema is:

<ul>
<li> timestamp
<li> duration (log)
<li> source computer
<li> source port
<li> destination computer
<li> destination port
<li> protocol
<li> nr packets (log)
<li> nr bytes (log)
</ul>

In [20]:
# load box/port info from json:

import json
import re

boxdata = json.load(open("data/boxport_data.json", 'r'))

def jsonIntKeys(x):
    if isinstance(x, dict):
            return {int(k):str(v) for k,v in x.items()}
    return x

def jsonIntVals(x):
    if isinstance(x, dict):
            return {str(k):int(v) for k,v in x.items()}
    return x

box_index = jsonIntVals( boxdata['box_indices'] )
index_box = jsonIntKeys( boxdata['indices_box'] )
port_index = jsonIntVals( boxdata['port_indices'] )
index_port = jsonIntKeys( boxdata['indices_port'] )

bbox = set([re.sub('[\"\n]+', "", s) for s in box_index.keys()])

In [21]:
# set RNN input vector sizes:

lg_duration = 7
lg_packets = 22
lg_bytes = 32
nr_proto = 4
nr_box = len(index_box)   # 10109
nr_port = len(index_port) #   326

# Import box embedding

In [15]:
# create dict from box name to 4-dimensional coords:

infile = "data/box_coords.csv"

fp = open(infile, 'r')
boxcoords = {line[0]: [float(line[i]) for i in range(1,5)] for line in csv.reader(fp)}
fp.close()

In [23]:
import numpy as np

comp_embed = np.zeros([nr_box, 4])

for b in boxcoords.keys():
    comp_embed[box_index[b],:] = boxcoords[b]

# RNN architecture

In [31]:
import sys
sys.setrecursionlimit(10000) # needed to prevent query-time bug...

from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.models import Model
from keras.engine.topology import merge
from keras.utils.visualize_util import plot

# hyperparameters:
nhidden = [512, 512]
unroll = 32
embed_dim = 4
dropout_W = 0.2 # input gates
dropout_U = 0.2 # recurrent connections

# netflow inputs:
src_input = Input(shape=(unroll,), dtype='int32', name='src_input')
dst_input = Input(shape=(unroll,), dtype='int32', name='dst_input')
src_pt_input = Input(shape=(unroll,), dtype='int32', name='src_pt')
dst_pt_input = Input(shape=(unroll,), dtype='int32', name='dst_pt')
proto_input = Input(shape=(unroll,), dtype='int32', name='proto')
duration_input = Input(shape=(unroll,), dtype='int32', name='duration')
packets_input = Input(shape=(unroll,), dtype='int32', name='packets')
bytes_input = Input(shape=(unroll,), dtype='int32', name='bytes')

# shared embedding for computer feeds:
comp_encoding = Embedding(output_dim=embed_dim, input_dim=nr_box, input_length=unroll, weights=[comp_embed], trainable=False)
src = comp_encoding(src_input)
dst = comp_encoding(dst_input)

# other embeddings:
src_pt = Embedding(output_dim=128, input_dim=nr_port, input_length=unroll)(src_pt_input)
dst_pt = Embedding(output_dim=128, input_dim=nr_port, input_length=unroll)(dst_pt_input)
proto = Embedding(output_dim=2, input_dim=nr_proto, input_length=unroll)(proto_input)
duration = Embedding(output_dim=4, input_dim=lg_duration, input_length=unroll)(duration_input)
packets = Embedding(output_dim=8, input_dim=lg_packets, input_length=unroll)(packets_input)
bytes = Embedding(output_dim=16, input_dim=lg_bytes, input_length=unroll)(bytes_input)

# merge:
data_merged = merge([src_pt, dst_pt, proto, duration, packets, bytes], mode='concat')

# add src computer for next time-step, as a query stream to train on:
next_src = Input(shape=(unroll,), dtype='int32', name='next_src')
query = comp_encoding(next_src)

# pass data and query to RNN layers:
inner = merge([data_merged, src, dst, query], mode='concat')
for i in range(len(nhidden)-1):
    inner = LSTM(nhidden[i], return_sequences=True, dropout_U=dropout_U, dropout_W=dropout_W)(inner)
inner = LSTM(nhidden[-1], return_sequences=False, dropout_U=dropout_U, dropout_W=dropout_W)(inner)
inner = Dropout(dropout_W)(inner)

# add softmax outputs:
proto_output = Dense(4, activation='softmax', name='proto_output')(inner)
duration_output = Dense(lg_duration, activation='softmax', name='duration_output')(inner)
packets_output = Dense(lg_packets, activation='softmax', name='packets_output')(inner)
bytes_output = Dense(lg_bytes, activation='softmax', name='bytes_output')(inner)
src_port_output = Dense(nr_port, activation='softmax', name='src_port_output')(inner)
dst_port_output = Dense(nr_port, activation='softmax', name='dst_port_output')(inner)

# add dst computer output:
next_dst = Dense(embed_dim, activation='relu', name='next_dst')(inner)

# put it all together:
model = Model(input=[src_input,
                     dst_input,
                     src_pt_input,
                     dst_pt_input,
                     proto_input,
                     duration_input,
                     packets_input, 
                     bytes_input,
                     next_src], 
              output=[proto_output,
                      duration_output,
                      packets_output,
                      bytes_output,
                      src_port_output,
                      dst_port_output,
                      next_dst])
model.compile(optimizer='rmsprop', 
              loss=['categorical_crossentropy' for i in range(6)] + ['mse'],
              loss_weights=[0.5, 1., 1., 1., 2., 2., 4.])

# ...and summarise:
model.summary()
plot(model, to_file='model.png')

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
bytes (InputLayer)                 (None, 32)          0                                            
____________________________________________________________________________________________________
dst_pt (InputLayer)                (None, 32)          0                                            
____________________________________________________________________________________________________
duration (InputLayer)              (None, 32)          0                                            
____________________________________________________________________________________________________
packets (InputLayer)               (None, 32)          0                                            
___________________________________________________________________________________________

# Prepare training sequence

In [58]:
# inputs:

import numpy as np

# set nr training points:
N = len(flowdata) - 1
N = 1000

train = np.array(flowdata[:N+1])

def make_input_vectors(z, mx, unroll=unroll):
    n = len(z)
    X = np.zeros((n - unroll, unroll), dtype='int32')
    for i in range(n - unroll):
        X[i,:] = z[i: i + unroll]
    return X

src_in = make_input_vectors(train[range(N), 2], nr_box)
dst_in = make_input_vectors(train[range(N), 4], nr_box)
src_pt_in = make_input_vectors(train[range(N), 3], nr_port)
dst_pt_in = make_input_vectors(train[range(N), 5], nr_port)
proto_in = make_input_vectors(train[range(N), 6], nr_proto)
duration_in = make_input_vectors(train[range(N), 1], lg_duration)
packets_in = make_input_vectors(train[range(N), 7], lg_packets)
bytes_in = make_input_vectors(train[range(N),8], lg_bytes)
next_src_in = make_input_vectors(train[range(1,N+1),2], nr_box)

input = [src_in, dst_in, src_pt_in, dst_pt_in, proto_in, duration_in, packets_in, bytes_in, next_src_in]

In [59]:
# targets:

def make_onehot(x, n):
    N = len(x)-unroll
    out = np.zeros([N,n])
    for i in range(1,N):
        out[i, x[i]] = 1
    return out

proto_tgt = make_onehot(train[range(1,N+1), 6], nr_proto)
duration_tgt = make_onehot(train[range(1,N+1), 1], lg_duration)
packets_tgt = make_onehot(train[range(1,N+1), 7], lg_packets)
bytes_tgt = make_onehot(train[range(1,N+1),8], lg_bytes)
src_pt_tgt = make_onehot(train[range(1,N+1), 3], nr_port)
dst_pt_tgt = make_onehot(train[range(1,N+1), 5], nr_port)

def make_nextdst(x, cmp_code):
    N = len(x)-unroll
    out = np.zeros([N, embed_dim])
    for i in range(N):
        out[i,:] = cmp_code[x[i],:]
    return out

dst_tgt = make_nextdst(train[range(1,N+1), 4], comp_embed)
target = [proto_tgt, duration_tgt, packets_tgt, bytes_tgt, src_pt_tgt, dst_pt_tgt, dst_tgt]

# Testing the architecture (optional)

In [42]:
# theano:

test = [x[:100] for x in input]
output = model.predict(test)
wts = model.get_weights()

print([y.shape for y in output])
print(output[0][1:10])

[(68, 4), (68, 7), (68, 22), (68, 32), (68, 326), (68, 326), (68, 4)]
[[  2.19199564e-02   8.41898501e-01   1.34521514e-01   1.66002382e-03]
 [  2.14816034e-02   8.51683855e-01   1.25184983e-01   1.64957228e-03]
 [  1.58133283e-02   8.52252007e-01   1.30265236e-01   1.66942494e-03]
 [  1.46434307e-02   8.24254453e-01   1.59528673e-01   1.57344039e-03]
 [  3.29837538e-02   9.19152975e-01   4.76046912e-02   2.58604705e-04]
 [  2.66646165e-02   9.46646631e-01   2.64343861e-02   2.54380633e-04]
 [  3.62941884e-02   9.34367657e-01   2.90191695e-02   3.19014420e-04]
 [  2.05556359e-02   8.14259052e-01   1.63295239e-01   1.89006177e-03]
 [  2.37861760e-02   8.25625598e-01   1.48486331e-01   2.10188935e-03]]


In [23]:
# tensorflow:

output = model(input)
output

In [43]:
# of particular interest is the shared computer encoding, which will also be needed in the loss function:
wts = model.get_weights()
for w in wts: print(w.shape)

(326, 128)
(4, 2)
(7, 4)
(22, 8)
(32, 16)
(326, 128)
(10109, 4)
(298, 512)
(512, 512)
(512,)
(298, 512)
(512, 512)
(512,)
(298, 512)
(512, 512)
(512,)
(298, 512)
(512, 512)
(512,)
(512, 512)
(512, 512)
(512,)
(512, 512)
(512, 512)
(512,)
(512, 512)
(512, 512)
(512,)
(512, 512)
(512, 512)
(512,)
(512, 32)
(32,)
(512, 326)
(326,)
(512, 7)
(7,)
(512, 4)
(4,)
(512, 22)
(22,)
(512, 4)
(4,)
(512, 326)
(326,)


In [54]:
# check the computer embedding:

wts = model.get_weights()
print(wts[6][0])
print(comp_embed[0])

[  6.40943575  -0.33822641  49.54575729  39.56111526]
[  6.4094359   -0.33822642  49.54575826  39.56111665]


# Training the model

In [60]:
# model fitting -- after each epoch, reset the target dst using the updated computer embedding matrix:
# for 10^5 records, takes 50 mins/epoch

#nr_epochs = 5

#for i in range(nr_epochs):
#    cmp_code = model.get_weights()[6]
#    target[6] = make_nextdst(train[range(1,N+1), 4], cmp_code)
#    model.fit(input, target, nb_epoch=1, verbose=1)

In [61]:
nr_epochs = 5

model.fit(input, target, nb_epoch=nr_epochs, verbose=1)

Epoch 1/5
968/968 [==============================] - 25s - loss: 2528.7871 - proto_output_loss: 0.6981 - duration_output_loss: 1.6187 - packets_output_loss: 1.7694 - bytes_output_loss: 2.4323 - src_port_output_loss: 1.5533 - dst_port_output_loss: 1.4473 - next_dst_loss: 629.1541    
Epoch 2/5
968/968 [==============================] - 26s - loss: 2508.8484 - proto_output_loss: 0.6906 - duration_output_loss: 1.5745 - packets_output_loss: 1.7365 - bytes_output_loss: 2.3709 - src_port_output_loss: 1.4319 - dst_port_output_loss: 1.3781 - next_dst_loss: 624.3003    
Epoch 3/5
968/968 [==============================] - 26s - loss: 2498.0047 - proto_output_loss: 0.6638 - duration_output_loss: 1.5705 - packets_output_loss: 1.7405 - bytes_output_loss: 2.3517 - src_port_output_loss: 1.4060 - dst_port_output_loss: 1.3511 - next_dst_loss: 621.6240    
Epoch 4/5
968/968 [==============================] - 25s - loss: 2475.5485 - proto_output_loss: 0.6827 - duration_output_loss: 1.5754 - packets_outp